In [ ]:
'''
Generate panels for figure 9 

Maximilian Eggl
Nov 2024
'''


import sys
    
from ExtFuncs import *
%load_ext autoreload
%autoreload 2

import scipy.stats as stats

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)

from sklearn.preprocessing import StandardScaler


plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import davies_bouldin_score
from matplotlib.markers import MarkerStyle
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler

In [ ]:
datDir = './Dat/'
ColorCode = {'D1Nac':np.array([117/255,20/255,124/255]),'D2Nac':np.array([241/255,164/255,84/255]),
             'mPFC':np.array([86/255,187/255,126/255]),'Sus':np.array((0.9682, 0.7208441, 0.612293)),
            'Res':np.array((0.705673158, 0.01555616, 0.150232812)),'nForg':np.array((0.6672529243333334, 0.779176457, 0.992959213)),
            'Forg':np.array((0.2298057, 0.298717966, 0.753683153))}

def PlotSig(left,right,height,starAdjust=0.02,Text=None):
    plt.plot([left,right],[height]*2,c='k')
    if(Text is None):
        plt.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c='k')
    else:
        plt.text((left+right)/2,height+starAdjust,Text,ha='center')
        
def PlotBoxScat(A,col,pos,lab=None,ax= None):
    if(ax is None):
        plt.boxplot(A,
                patch_artist=True,
                boxprops=dict(facecolor='none', 
                              color=col),
                capprops=dict(color=col),
                whiskerprops=dict(color=col),
                flierprops=dict(color=col, markeredgecolor=
                                col),
                medianprops=dict(color='g'),
                positions=[pos],
                            showfliers=False
                )  
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
    else:
        ax.boxplot(A,
                patch_artist=True,
                boxprops=dict(facecolor='none', 
                              color=col),
                capprops=dict(color=col),
                whiskerprops=dict(color=col),
                flierprops=dict(color=col, markeredgecolor=
                                col),
                medianprops=dict(color='g'),
                positions=[pos],
                            showfliers=False
                )  
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
def PlotViolScat(A,col,pos,lab=None,ax= None,hatch=False):
    if(ax is None):
        vp = plt.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        if(hatch):
            vp['bodies'][0].set_hatch('//')
    else:
        vp = ax.violinplot(A,
                positions=[pos],widths=0.2
                )  
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        
def z_score(X):
    # X: ndarray, shape (n_features, n_samples)
    ss = StandardScaler(with_mean=True, with_std=True)
    Xz = ss.fit_transform(X.T).T
    return Xz

In [ ]:
def GetGPIO(datDir):
    for file in os.listdir(datDir):
        if file.endswith(".csv"):
            GDat = pd.read_csv(datDir+'/'+file)
            G1 = GDat[GDat[' Channel Name'] == ' GPIO-1']
            G2 = GDat[GDat[' Channel Name'] == ' GPIO-2']
            G3 = GDat[GDat[' Channel Name'] == ' GPIO-3']
            try:
                Tmax = G1['Time (s)'].max()
            except Exception as e:
                print(e)
                EndTime,StartTime,Tmax = math.nan,math.nan,math.nan
            return Tmax,G2,G3
def GetDat_loc(Batch,Expts,datDir,FreezeTime=50,eFlag = 0):
    print('='*20)
    print('Doing '+Batch)
    DatNames  = []
    Dat       = []
    Times     = []
    ShockTimes = []
    TimeCalcDat = []
    CalcDat = []
    RawDat  = []
    Lights  = []
    for file in tqdm(os.listdir(datDir+Batch)):
        print(file)
        if os.path.isdir(datDir+Batch+'/'+file):  
            newDir = datDir+Batch+'/'+file+'/'
            DatNames.append(file)
            T = []
            d = []
            st = []
            for file2 in np.sort(os.listdir(newDir)):
                if(os.path.isdir(newDir+file2) and file2+'/' in Expts):
                    f,t,s = GetFreezingNew(newDir+file2+'/')
                    f2 = FilterFreezing(f[1],FreezeTime)
                    f.append(f2)
                    d.append(f)
                    T.append(t)
                    st.append(s)
            Dat.append(d)
            Times.append(T)
            ShockTimes.append(st)
            
            newDir = datDir+Batch+'/'+file+'/'
            DatNames.append(file)
            T = []
            d = []
            st = []
            litLights = []
            for file2 in np.sort(os.listdir(newDir)):
                if(os.path.isdir(newDir+file2) and file2+'/' in Expts):
                    t,g2,g3 = GetGPIO(newDir+file2+'/')
                    T.append(t)
                    litLights.append([g2,g3])
            Lights.append(litLights)
            MatDir =datDir+Batch+'/'+file+'/Data_Miniscope_PP.mat'
            NDat = read_mat(MatDir)
            sDat,cDat,cRaw = PrepData(NDat)
            CalcDat.append([cDat[0],cDat[-2]]) 
            RawDat.append([cRaw[0],cRaw[-2]]) 
            lens = [len(cDat[0][0]),len(cDat[-2][0])]
        
            TimeCalcDat.append([np.linspace(0,tm,l) for tm,l in zip(T,lens)])
    return Dat,np.array(Times),DatNames,TimeCalcDat,CalcDat,RawDat,Lights

In [ ]:
labelsMPC = np.array([1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])
labelsNAD1 = np.array([1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])
labelsNAD2 = np.array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [ ]:
DatA,TimesA,DatNamesA,TimeCalcDatA,CalcDatA,ShockTimesA,cRawA = GetDat('Batch A 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)
DatB,TimesB,DatNamesB,TimeCalcDatB,CalcDatB,ShockTimesB,cRawB = GetDat('Batch B 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)

DatMPC = DatA+DatB
TimesMPC = np.vstack([TimesA,TimesB])
DatNamesMPC = DatNamesA + DatNamesB
TimeCalcDatMPC = TimeCalcDatA+TimeCalcDatB
CalcDatMPC = CalcDatA+CalcDatB
ShockTimesMPC = np.vstack([ShockTimesA,ShockTimesB])

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,rawC = GetDat('Batch C 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,rawD = GetDat('Batch D 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)
DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = rawC+rawD

In [ ]:
DatE,TimesE,DatNamesE,TimeCalcDatE,CalcDatE,ShockTimesE,rawE = GetDat('Batch E 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=0)
DatF,TimesF,DatNamesF,TimeCalcDatF,CalcDatF,ShockTimesF,rawF = GetDat('Batch F 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=0)
DatNAD2 = DatE+DatF
TimesNAD2 = np.vstack([TimesE,TimesF])
DatNamesNAD2 = DatNamesE + DatNamesF
TimeCalcDatNAD2 = TimeCalcDatE+TimeCalcDatF
CalcDatNAD2 = CalcDatE+CalcDatF
ShockTimesNAD2 = np.vstack([ShockTimesE,ShockTimesF])
RawDatNAD2 = rawE+rawF

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    TC = TimeCalcDatMPC[j][i]
    ST = ShockTimesMPC[j][i]
    Order = np.zeros(Calc.shape[0])
    Order2 = np.zeros(Calc.shape[0])
    Fire0 = Calc[:,TC<ST[0]].mean(axis=1)==0.0
    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order = np.zeros_like(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    FullOrder.append(Order)
    FullOrderA.append(Order2)
    
print('-'*10 + 'Done 1'+'-'*10)    
i = 1
FullOrder2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)
print('-'*10 + 'Done 2'+'-'*10)
i = 2
FullOrder3 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
print('-'*10 + 'Done 3'+'-'*10)
i = 3
FullOrder4 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
print('-'*10 + 'Done 4'+'-'*10)
TotalOrder = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrder,FullOrder2,FullOrder3,FullOrder4)]

In [ ]:
FullOrderD1 = []
FullOrderD1A = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD1.append(Order)
    FullOrderD1A.append(Order2)

print('1 done')

i = 1
FullOrderD12 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD1[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD1A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD12.append(Order)

print('2 done')

i = 2
FullOrderD13 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD13.append(Order)
    
print('3 done')

i = 3
FullOrderD14 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]<0,FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]>0,FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD14.append(Order)
    
print('4 done')

TotalOrderD1 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD1,FullOrderD12,FullOrderD13,FullOrderD14)]

In [ ]:
FullOrderD2 = []
FullOrderD2A = []
i=0
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    TC = TimeCalcDatNAD2[j][i]
    ST = ShockTimesNAD2[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD2.append(Order)
    FullOrderD2A.append(Order2)

print('1 done')

i = 1
FullOrderD22 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD2[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD2A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD22.append(Order)

print('2 done')

i = 2
FullOrderD23 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD23.append(Order)
    
print('3 done')

i = 3
FullOrderD24 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]<0,FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]>0,FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD24.append(Order)
    
print('4 done')

TotalOrderD2 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD2,FullOrderD22,FullOrderD23,FullOrderD24)]

In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch A 2022 mPFC',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch B 2022 mPFC',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

N = 200

In [ ]:
i=0
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrder[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        

    minT = max((np.min(TD),np.min(T)))
    maxT = min((np.max(TD),np.max(T)))
    
    mask1 = np.logical_and(T>minT,T<maxT)
    mask2 = np.logical_and(TD>minT,TD<maxT)

    Calc = Calc[:,mask1]
    T    = T[mask1]
    
    f = sp.interpolate.interp1d(TD,D)
    D_int = f(T)
    Corrs = []
    for C in Calc:
        Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpre= np.array([np.nanmean(C) for C in CorrShock])
KOpre = np.array([np.nanmean(C) for C in CorrOShock])


i=1
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrder[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        
    if(sum(D)==0):
        Corrs = np.zeros(len(Calc))
    else:
        minT = max((np.min(TD),np.min(T)))
        maxT = min((np.max(TD),np.max(T)))

        mask1 = np.logical_and(T>minT,T<maxT)
        mask2 = np.logical_and(TD>minT,TD<maxT)

        Calc = Calc[:,mask1]
        T    = T[mask1]

        f = sp.interpolate.interp1d(TD,D)
        D_int = f(T)
        Corrs = []
        for C in Calc:
            Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpost = np.array([np.nanmean(C) for C in CorrShock])
KOpost = np.array([np.nanmean(C) for C in CorrOShock])

In [ ]:
A = np.array([FilterFreezing(DAH[j][0][1],0.3) for j in range(len(CH))],dtype=object)
B = np.array([FilterFreezing(DAH[j][1][1],0.3) for j in range(len(CH))],dtype=object)
Hab1 = [100*sum(a)/len(a) for a in A]
Hab2 = [100*sum(b)/len(b) for b in B]

PlotViolScat(Hab1,np.clip(ColorCode['D1Nac'],0,1),1,'G1')
PlotViolScat(Hab2,np.clip(ColorCode['D1Nac'],0,1),2)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)
plt.show()

Hab1_Rp = [100*sum(a)/len(a) for a in A[labelsMPC==2]]
Hab2_Rp = [100*sum(b)/len(b) for b in B[labelsMPC==2]]

Hab1_Rm = [100*sum(a)/len(a) for a in A[labelsMPC==1]]
Hab2_Rm = [100*sum(b)/len(b) for b in B[labelsMPC==1]]

PlotViolScat(Hab1_Rp,np.clip(ColorCode['D1Nac']-0.1,0,1),1,lab='R+')
PlotViolScat(Hab2_Rp,np.clip(ColorCode['D1Nac']-0.1,0,1),2)
PlotViolScat(Hab1_Rm,np.clip(ColorCode['D1Nac']+0.1,0,1),1.2)
PlotViolScat(Hab2_Rm,np.clip(ColorCode['D1Nac']+0.1,0,1),2.2,lab='R-')
plt.legend(fontsize=24)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)

In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch C 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch D 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

In [ ]:
i=0
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrderD1[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        

    minT = max((np.min(TD),np.min(T)))
    maxT = min((np.max(TD),np.max(T)))
    
    mask1 = np.logical_and(T>minT,T<maxT)
    mask2 = np.logical_and(TD>minT,TD<maxT)

    Calc = Calc[:,mask1]
    T    = T[mask1]
    
    f = sp.interpolate.interp1d(TD,D)
    D_int = f(T)
    Corrs = []
    for C in Calc:
        Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpre= np.array([np.nanmean(C) for C in CorrShock])
KOpre = np.array([np.nanmean(C) for C in CorrOShock])


i=1
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrderD1[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        
    if(sum(D)==0):
        Corrs = np.zeros(len(Calc))
    else:
        minT = max((np.min(TD),np.min(T)))
        maxT = min((np.max(TD),np.max(T)))

        mask1 = np.logical_and(T>minT,T<maxT)
        mask2 = np.logical_and(TD>minT,TD<maxT)

        Calc = Calc[:,mask1]
        T    = T[mask1]

        f = sp.interpolate.interp1d(TD,D)
        D_int = f(T)
        Corrs = []
        for C in Calc:
            Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpost = np.array([np.nanmean(C) for C in CorrShock])
KOpost = np.array([np.nanmean(C) for C in CorrOShock])

In [ ]:
A = np.array([FilterFreezing(DAH[j][0][1],0.3) for j in range(len(CH))],dtype=object)
B = np.array([FilterFreezing(DAH[j][1][1],0.3) for j in range(len(CH))],dtype=object)
Hab1 = [100*sum(a)/len(a) for a in A]
Hab2 = [100*sum(b)/len(b) for b in B]

PlotViolScat(Hab1,np.clip(ColorCode['D2Nac'],0,1),1,'G1')
PlotViolScat(Hab2,np.clip(ColorCode['D2Nac'],0,1),2)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)

Hab1_Rp = [100*sum(a)/len(a) for a in A[labelsNAD1==2]]
Hab2_Rp = [100*sum(b)/len(b) for b in B[labelsNAD1==2]]

Hab1_Rm = [100*sum(a)/len(a) for a in A[labelsNAD1==1]]
Hab2_Rm = [100*sum(b)/len(b) for b in B[labelsNAD1==1]]

PlotViolScat(Hab1_Rp,np.clip(ColorCode['D2Nac']-0.1,0,1),1,lab='R+')
PlotViolScat(Hab2_Rp,np.clip(ColorCode['D2Nac']-0.1,0,1),2)
PlotViolScat(Hab1_Rm,np.clip(ColorCode['D2Nac']+0.1,0,1),1.2)
PlotViolScat(Hab2_Rm,np.clip(ColorCode['D2Nac']+0.1,0,1),2.2,lab='R-')
plt.legend(fontsize=24)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)

plt.plot([1,2],[50,50],'k')
x = np.array([1.48,1.52])
y = [52]*2
plt.plot(x, y, marker=(6, 2, 0), ms=10, c='k', linestyle='') 


In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch E 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch F 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

In [ ]:
i=0
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrderD2[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        

    minT = max((np.min(TD),np.min(T)))
    maxT = min((np.max(TD),np.max(T)))
    
    mask1 = np.logical_and(T>minT,T<maxT)
    mask2 = np.logical_and(TD>minT,TD<maxT)

    Calc = Calc[:,mask1]
    T    = T[mask1]
    
    f = sp.interpolate.interp1d(TD,D)
    D_int = f(T)
    Corrs = []
    for C in Calc:
        Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpre= np.array([np.nanmean(C) for C in CorrShock])
KOpre = np.array([np.nanmean(C) for C in CorrOShock])


i=1
CorrShock = []
CorrAShock = []
CorrOShock = []
for j in range(len(CH)):
    Calc = CH[j][i]
    TO   = TotalOrderD2[j]
    Calc = Calc
    T = TCH[j][i]   
    D = FilterFreezing(DAH[j][i][1],2)
    TD = DAH[j][i][0]        
    if(sum(D)==0):
        Corrs = np.zeros(len(Calc))
    else:
        minT = max((np.min(TD),np.min(T)))
        maxT = min((np.max(TD),np.max(T)))

        mask1 = np.logical_and(T>minT,T<maxT)
        mask2 = np.logical_and(TD>minT,TD<maxT)

        Calc = Calc[:,mask1]
        T    = T[mask1]

        f = sp.interpolate.interp1d(TD,D)
        D_int = f(T)
        Corrs = []
        for C in Calc:
            Corrs.append(np.corrcoef(D_int,C)[0,1])
    CorrShock.append(np.array(Corrs)[TO])
    CorrOShock.append(np.array(Corrs)[~TO])
    
Kpost = np.array([np.nanmean(C) for C in CorrShock])
KOpost = np.array([np.nanmean(C) for C in CorrOShock])

In [ ]:
A = np.array([FilterFreezing(DAH[j][0][1],0.3) for j in range(len(CH))],dtype=object)
B = np.array([FilterFreezing(DAH[j][1][1],0.3) for j in range(len(CH))],dtype=object)
Hab1 = [100*sum(a)/len(a) for a in A]
Hab2 = [100*sum(b)/len(b) for b in B]

PlotViolScat(Hab1,np.clip(ColorCode['mPFC'],0,1),1,'G1')
PlotViolScat(Hab2,np.clip(ColorCode['mPFC'],0,1),2)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)
plt.show()

Hab1_Rp = [100*sum(a)/len(a) for a in A[labelsNAD2==2]]
Hab2_Rp = [100*sum(b)/len(b) for b in B[labelsNAD2==2]]

Hab1_Rm = [100*sum(a)/len(a) for a in A[labelsNAD2==1]]
Hab2_Rm = [100*sum(b)/len(b) for b in B[labelsNAD2==1]]

PlotViolScat(Hab1_Rp,np.clip(ColorCode['mPFC']-0.1,0,1),1,lab='R+')
PlotViolScat(Hab2_Rp,np.clip(ColorCode['mPFC']-0.1,0,1),2)
PlotViolScat(Hab1_Rm,np.clip(ColorCode['mPFC']+0.1,0,1),1.2)
PlotViolScat(Hab2_Rm,np.clip(ColorCode['mPFC']+0.1,0,1),2.2,lab='R-')
plt.legend(fontsize=24)
plt.xticks([1,2],['HB1','HB2'])
plt.ylabel('Time spent freezing \n [\%]',fontsize=32)